In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector as selector
from sklearn.pipeline import Pipeline
import joblib
from sklearn.preprocessing import LabelEncoder

In [2]:
# Load your CSV data
data = pd.read_csv('cleaned_data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65752 entries, 0 to 65751
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     65752 non-null  int64  
 1   Type                           65752 non-null  object 
 2   Days for shipping (real)       65752 non-null  int64  
 3   Days for shipment (scheduled)  65752 non-null  int64  
 4   Benefit per order              65752 non-null  float64
 5   Sales per customer             65752 non-null  float64
 6   Delivery Status                65752 non-null  object 
 7   Late_delivery_risk             65752 non-null  int64  
 8   Category Id                    65752 non-null  int64  
 9   Category Name                  65752 non-null  object 
 10  Department city                65752 non-null  object 
 11  Department country             65752 non-null  object 
 12  Customer Email                 65752 non-null 

In [4]:
data.head()

,Unnamed: 0,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [5]:
X=data[['Category Name','Customer Zipcode','Customer Street','Customer Id','Customer Segment','Department Id','Order City','Order Country','Order Item Quantity','Order Customer Id','order date (DateOrders)','Order Id','Order Item Cardprod Id','Order Region','Order State','Product Card Id','Product Name','Product Price','Shipping Mode']]
X

,Category Name,Customer Zipcode,Customer Street,Customer Id,Customer Segment,Department Id,Order City,Order Country,Order Item Quantity,Order Customer Id,order date (DateOrders),Order Id,Order Item Cardprod Id,Order Region,Order State,Product Card Id,Product Name,Product Price,Shipping Mode
0,Sporting Goods,725.0,5365 Noble Nectar Island,20755,Consumer,2,Bekasi,Indonesia,1,20755,1/31/2018 22:56,77202,1360,Southeast Asia,Java Occidental,1360,Smart watch,327.750000,Standard Class
1,Sporting Goods,725.0,2679 Rustic Loop,19492,Consumer,2,Bikaner,India,1,19492,1/13/2018 12:27,75939,1360,South Asia,Rajastán,1360,Smart watch,327.750000,Standard Class
2,Sporting Goods,95125.0,8510 Round Bear Gate,19491,Consumer,2,Bikaner,India,1,19491,1/13/2018 12:06,75938,1360,South Asia,Rajastán,1360,Smart watch,327.750000,Standard Class
3,Sporting Goods,90027.0,3200 Amber Bend,19490,Home Office,2,Townsville,Australia,1,19490,1/13/2018 11:45,75937,1360,Oceania,Queensland,1360,Smart watch,327.750000,Standard Class
4,Sporting Goods,725.0,8671 Iron Anchor Corners,19489,Corporate,2,Townsville,Australia,1,19489,1/13/2018 11:24,75936,1360,Oceania,Queensland,1360,Smart watch,327.750000,Standard Class
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65747,Fishing,725.0,6567 High Lagoon Heath,5857,Consumer,7,Ho Chi Minh City,Vietnam,1,5857,1/26/2016 19:25,26773,1004,Southeast Asia,Ciudad Ho Chi Minh,1004,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,First Class
65748,Fishing,43228.0,5593 Round Lane,9230,Consumer,7,Ezhou,China,1,9230,1/22/2016 6:49,26463,1004,Eastern Asia,Hubei,1004,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,Standard Class
65749,Fishing,30058.0,263 Harvest Wood,4618,Corporate,7,Tawau,Malasia,1,4618,1/21/2016 2:47,26383,1004,Southeast Asia,Sabah,1004,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,Second Class
65750,Fishing,725.0,1914 Blue Autumn Harbour,989,Corporate,7,Semarang,Indonesia,1,989,1/20/2016 7:10,26327,1004,Southeast Asia,Java Central,1004,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,Standard Class


In [6]:
# Assuming 'data' is your DataFrame
X = data.iloc[:, :-1]  # Selecting all rows and all columns except the last one

# 'X' now contains all columns except the last one


In [7]:
# Assuming X contains your feature columns and y contains the target variable (Shipping Mode)
# Split the data into features (X) and the target variable (y)
X = data.drop(columns=['Shipping Mode'])  # Assuming 'Shipping Mode' is the target column
y = data['Shipping Mode']

# Encoding the target variable using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [8]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Identify numerical and categorical columns automatically
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Preprocessing steps for numerical and categorical columns
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Define the Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Create a pipeline with preprocessing and the model
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Train the model using the pipeline
pipeline.fit(X_train, y_train)

# Make predictions on test data
y_pred = pipeline.predict(X_test)

# Convert back predicted labels to their original string form
y_pred_original = label_encoder.inverse_transform(y_pred)

# Calculate accuracy for each shipping mode
shipping_modes = ['Standard Class','First Class', 'Second Class', 'Same Day']
for mode in shipping_modes:
    mode_index = label_encoder.classes_.tolist().index(mode)
    y_test_mode = (y_test == mode_index)
    y_pred_mode = (y_pred == mode_index)
    accuracy_mode = accuracy_score(y_test_mode, y_pred_mode)
    print(f"Accuracy for '{mode}': {accuracy_mode}")

# Save the model to a file
joblib.dump(pipeline, 'random_forest_model.pkl')


/Users/imandikarunaratne/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['Product Description']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/imandikarunaratne/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['Product Description']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


Accuracy for 'Standard Class': 0.9752110105695384
Accuracy for 'First Class': 0.9692799026689985
Accuracy for 'Second Class': 0.9777203254505361
Accuracy for 'Same Day': 0.9690517831343624


['random_forest_model.pkl']

In [10]:
from sklearn.svm import SVC

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Identify numerical and categorical columns automatically
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Preprocessing steps for numerical and categorical columns
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Define the SVM classifier
model = SVC(kernel='linear', C=1.0, random_state=42)  # Using a linear kernel

# Create a pipeline with preprocessing and the model
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Train the model using the pipeline
pipeline.fit(X_train, y_train)

# Make predictions on test data
y_pred = pipeline.predict(X_test)

# Convert back predicted labels to their original string form
y_pred_original = label_encoder.inverse_transform(y_pred)

# Calculate accuracy for each shipping mode
shipping_modes = ['Standard Class', 'First Class', 'Second Class', 'Same Day']
for mode in shipping_modes:
    mode_index = label_encoder.classes_.tolist().index(mode)
    y_test_mode = (y_test == mode_index)
    y_pred_mode = (y_pred == mode_index)
    accuracy_mode = accuracy_score(y_test_mode, y_pred_mode)
    print(f"Accuracy for '{mode}': {accuracy_mode}")

# Save the model to a file
joblib.dump(pipeline, 'svm_model.pkl')


/Users/imandikarunaratne/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['Product Description']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/imandikarunaratne/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['Product Description']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


Accuracy for 'Standard Class': 1.0
Accuracy for 'First Class': 1.0
Accuracy for 'Second Class': 1.0
Accuracy for 'Same Day': 1.0


['svm_model.pkl']

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Identify numerical and categorical columns automatically
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Preprocessing steps for numerical and categorical columns
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Define the XGBoost classifier
model = XGBClassifier(random_state=42)

# Create a pipeline with preprocessing and the model
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Train the model using the pipeline
pipeline.fit(X_train, y_train)

# Make predictions on test data
y_pred = pipeline.predict(X_test)

# Convert back predicted labels to their original string form
y_pred_original = label_encoder.inverse_transform(y_pred)

# Calculate accuracy for each shipping mode
shipping_modes = ['Standard Class', 'First Class', 'Second Class', 'Same Day']
for mode in shipping_modes:
    mode_index = label_encoder.classes_.tolist().index(mode)
    y_test_mode = (y_test == mode_index)
    y_pred_mode = (y_pred == mode_index)
    accuracy_mode = accuracy_score(y_test_mode, y_pred_mode)
    print(f"Accuracy for '{mode}': {accuracy_mode}")

# Save the model to a file
joblib.dump(pipeline, 'xgboost_model.pkl')


/Users/imandikarunaratne/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['Product Description']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/Users/imandikarunaratne/anaconda3/lib/python3.11/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['Product Description']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


Accuracy for 'Standard Class': 1.0
Accuracy for 'First Class': 1.0
Accuracy for 'Second Class': 1.0
Accuracy for 'Same Day': 1.0


['xgboost_model.pkl']